# Lecture 6: Big Step Semantics For Expressions

We have thus far looked at the first eval function for `Expr`  that denote arithmetic expressions.

Recall their grammar from the previous lectures. We have simplified the grammer slightly:  plus, minus and multiplication operate just over two expressions at a time.

$$\begin{array}{rcll}
\textbf{Expr} & \rightarrow & Const(\textbf{Double}) \\
& |  & Ident(\textbf{Identifier}) \\
& | & Plus( \textbf{Expr}, \textbf{Expr})  \\
& | & Minus( \textbf{Expr}, \textbf{Expr}) \\
& | & Mult(\textbf{Expr}, \textbf{Expr}) \\
& | & Div(\textbf{Expr}, \textbf{Expr}) \\
& | & Log(\textbf{Expr}) \\
& | & Exp(\textbf{Expr}) \\
& | & Sine(\textbf{Expr}) \\
& | & Cosine(\textbf{Expr}) \\\\
\textbf{Double} & \rightarrow & \text{all double precision numbers in Scala}\\
\textbf{Identifier} & \rightarrow & [a-zA-Z][a-z\ A-Z\ 0-9\ \_]* & \text{Note: All strings that begin with letters}\\
&&& \text{a-z or A-Z and subsequently can contain a-z, A-Z, 0-9 or _ chars}
\end{array}$$

Next we provided a translation of the very same grammar into scala abstract syntax definitions.

In [3]:
sealed trait Expr
case class Const(f: Double) extends Expr 
case class Ident(s: String) extends Expr
case class Plus( e1: Expr, e2: Expr ) extends Expr
case class Minus(e1: Expr, e2: Expr) extends Expr
case class Mult(e1: Expr, e2: Expr ) extends Expr
case class Div(e1: Expr, e2: Expr) extends Expr
case class Log(e: Expr) extends Expr
case class Exp(e: Expr) extends Expr
case class Sine(e: Expr) extends Expr
case class Cosine(e: Expr) extends Expr

defined trait Expr
defined class Const
defined class Ident
defined class Plus
defined class Minus
defined class Mult
defined class Div
defined class Log
defined class Exp
defined class Sine
defined class Cosine

We wrote a simple and intuitive function `evalExpr` but how do we talk about what it does? A simple answer is that the code can be read by anyone who knows Scala and therefore can serve to document the meaning of expressions in terms of what they do. This is surely a path to disaster for more complicated languages. It is important therefore to use some kind of notation to explain how to understand the behavior of the `evalExpr` function at a high level and in a language independent fashion. We will use "big step" semantics to notate this process and see how the code we have written so far is simply a translation of the big step semantics into code.


## Big Step Semantics

Let us explain at a high level how an expression $ \log(1.0 + 2.0 * y)$ (note $\log$ is the natural log to the base $e$).

`Log(Plus(Const(1.0), Mult(Const(2.0), Ident("y"))))` 

under an environment that assignes `y` to `1.2` gets evaluated to a value `1.2237754316221157`.

Without worrying about the order in which things happen, let us act as if we are ourselves working this out.

- `Ident("y")` gets evaluated to `1.2`
- `Const(2.0)` gets evaluated to `2.0`
- `Mult(Const(2.0), Ident("y"))` can now get evaluated to `2.0 * 1.2 = 2.4` since each of its children have been evaluated
- `Const(1.0)` gets evaluated to `1.0`.
- `Plus(Const(1.0) , Mult(Const(2.0), Ident("y")))` can be evaluated to `1.0 + 2.4` and thus to 3.4
- `Log(...)` evaluates to $\log(3.4) = 1.223775...$.

What are the things we notice in this process?

- Every rule takes an expression and tells us what it evaluates to.
- The environment mapping variables to values is an input to this process and remains unchanged throughout.
- The result of every evaluation is a double precision number.
- Simple leaf expressions like `Const(f)` and `Identifier(y)` are evaluated directy to the double precision numbers either as the number `f` or whatever value the identifier `y` is mapped to in the given environment.
- For any other expression using construtors `Plus`, `Minus`, `Mult`, `Div`, ..., if the children evaluate to double precision numbers, then the expression itself is ready to evaluate by applying the appropriate arithmetic operator on the values of its children.

These observations are tedious to write down in english. We can provide a more succinct way to write it down using a notation:  $\mathbf{eval}(\texttt{e}, \sigma) = d$.
It is a way of succinctly saying that "under the enviroment $\sigma$, the expression $\texttt{e}$ evaluates to the value $d$". 

$\newcommand\eval{\mathbf{eval}}$
### Examples

- $ \eval(\texttt{Ident("y")}, \{y \mapsto 1.0 \})\ = 1.0 $
- $ \eval(\texttt{Mult(Const(1.2), Const(1.2))}, \{ y \mapsto 1.0 \} ) = 1.44 $.
- $ \eval(\texttt{Plus(Const(1.0) , Mult(Const(2.0), Ident("y")))}, \{y \mapsto 1.2 \})= 3.4 $


## Inference Rules

The value in the entire process is not in realizing individual instances such as 

$$ \eval\left(\texttt{Log(Plus(Const(1.0) , Mult(Const(2.0), Ident("y")))}, \{y \mapsto 1.2 \} \right) = 1.223775.. $$

but in specifying how one systematically arrives at the result. If that is understood, then writing an interpreter is a cinch. Also, the rules will specify exactly how we will go about the process.

Inference rules are always written like this.

$$\begin{array}{c}
\text{premises that must hold} \\
\hline
\text{conclusions that can be drawn} \\
\end{array}$$

It must be read as __assuming premises must hold, then conclusion must hold__.

As an example, let us see a rule for `Log`:

$$\begin{array}{c}
\eval(\texttt{e},\sigma) = c \\
\hline 
\eval(\texttt{Log(e)}, \sigma) = \log(c) \\
\end{array}\ \text{(Log)}$$

The rule says:
- Assume: "under the environment $\sigma$, some expression `e` evaluates to $c$"
- Conclude: "under the environment $\sigma$, the expression `Log(e)` evalutes to $\log(c)$"

#### Basic Inference Rules

Let us actually dive into some basic inference rules:

$$ \begin{array}{c}
\\
\hline
\eval\left(\texttt{Const(f)}, \sigma\right) = f \\
\end{array}\ \text{(Const)} $$

First note that the premises are empty, so no assumptions are needed for the conclusion.
The inference rule above simply says the following:
- Under any environment $\sigma$, the expression `Const(f)` evaluates to `f`.

Another basic inference rule is 

$$ \begin{array}{c}
\\
\hline
\eval(\texttt{Ident(s)}, \sigma) = \sigma(s)\\
\end{array}\ \text{(Variable)} $$

Under any environment $\sigma$, the expression `Ident(s)` evaluates to $\sigma(s)$. __But the reader may object__:
what is $\sigma$, what is $\sigma(s)$ and what do we do if $\sigma$ does not have a value for $s$. These
are great questions. To resolve them, we must say what an environment is.

### Environments

An environment is a partial function from names of identifiers to their values.
- Let $Domain(\sigma)$ be the set of all variables defined in an environment.
- Let $\sigma(s)$ be the value mapped to by identifier $s$ if $s \in Domain(\sigma)$.


### Basic Inferences (Continued)

Resuming where we left off, we will now provide a refined rule 

$$\begin{array}{c}
s \in \text{Domain}(\sigma)\\
\hline
\eval(\texttt{Ident(s)},\sigma) = \sigma(s) \\
\end{array}\ (\text{Ident})$$

Can you read this rule aloud for us?

- Premise: the identifier $s$ belongs to $\text{Domain}(\sigma)$
- Conclusion: $\eval(\texttt{Ident(s)},\sigma) = \sigma(s)$

What about a rule for  $s \not \in \text{Domain}(\sigma)$? Let us take a *raincheck* on itand get back to it in a short while.

### Compound Inference

Now that we have the basic rules for constants and identifiers, how do we proceed for the rest?

#### Plus

$$\begin{array}{c}
\eval(\texttt{e1}, \sigma) = c_1,\ \eval(\texttt{e2}, \sigma) =  c_2\\
\hline
\eval\left( \texttt{Plus(e1,e2)}, \sigma\right)  =  (c_1 + c_2) \\
\end{array} (\text{Plus}) $$


#### Minus

$$\begin{array}{c}
\eval(\texttt{e1}, \sigma) = c_1,\ \eval(\texttt{e2}, \sigma) =  c_2 \\
\hline
\eval\left(\texttt{Minus(e1, e2)}, \sigma\right) = (c_1 - c_2 ) \\
\end{array} (\text{Minus}) $$

#### Multiplication

$$\begin{array}{c}
\eval(\texttt{e1}, \sigma) = c_1,\ \eval(\texttt{e2}, \sigma) =  c_2 \\
\hline
\eval\left(\texttt{Mult(e1, e2)}, \sigma\right) = (c_1 \times c_2) \\
\end{array} (\text{Mult}) $$

#### Division

$$\begin{array}{c}
\eval(\texttt{e1}, \sigma) = c_1,\ \eval(\texttt{e2}, \sigma) =  c_2,\ \color{red}{c_2 \not= 0}\\
\hline
\eval\left( \texttt{Div(e1, e2)}, \sigma \right) = (\frac{c_1}{c_2} ) \\
\end{array} (\text{Div}) $$

Note the premise that $c_2 \not= 0$. What happens if $c_2 = 0$? We will need to specify that under error handling.


#### Other Rules
We already saw an example for `Log`. Let's refine it by adding the condition that the argument of `Log` must always be positive.
$$\begin{array}{c}
\eval\left( \texttt{e}, \sigma\right) = c,\ \color{red}{c > 0} \\
\hline 
\eval(\texttt{Log(e)}, \sigma) = \log(c) \\
\end{array}\ \text{(Log)}$$

Rather than a rule for each of  Exp, Sine, Cosine: we can provide a single "rule template"  -- a sort of macro that can be used for each of these rules? 

Before we do that let us define an association of each of these constructors with the functions they represent. Though it is obvious to us from the naming, it is not so to a computer. Therefore, we must pretend ignorance and specify the same.

$$ f_{\texttt{Exp}}(x) = e^x,\ f_{\texttt{Sine}}(x) = \sin(x),\ f_{\texttt{Cosine}}(x) = \cos(x) $$. 

$$\begin{array}{c}
\eval(\texttt{e}, \sigma) = c,\ T \in \{ \texttt{Exp}, \texttt{Sine}, \texttt{Cosine} \} \\
\hline
\eval\left( \texttt{T(e)}, \sigma \right) =  f_{\texttt{T}}(c)
\end{array} (\text{InBuilt-Function-Application})$$

## Handling Error

Semantics is necessary not just to define the correct cases, but also to inform us what to do if there is an error.
Often, we can distinguish between the types of errors that we would like to handle such as `DivideByZero`, `UndefinedVariable`, and `IllegalArgument`. Here, we will keep things simple and define just one error type __error__.

So far, expressions could just take a value that was a double precision. We now augment it to take either 
a double precision value or a special value called __error__.

The rules for producing double precision values have already been seen and recalled below once more.
We will augment these rules to say that there is no error involved. The changes to the premises are underlined.
They mostly involve saying things like $c_i \in \mathbb{R}$ (i.e, a value $c_i$ is a real number).


$$\begin{array}{c}
\eval(\texttt{e1}, \sigma) = c_1,\ \eval(\texttt{e2}, \sigma) =  c_2,\ \color{red}{c_1\in \mathbb{R}, c_2 \in \mathbb{R}}\\
\hline
\eval\left( \texttt{Plus(e1, e2)}, \sigma\right)  =  (c_1 + c_2 ) \\
\end{array} (\text{Plus}) $$

$$\begin{array}{c}
\eval(\texttt{e1}, \sigma) = c_1,\ \eval(\texttt{e2}, \sigma) =  c_2,\ \color{red}{c_1\in \mathbb{R}, c_2 \in \mathbb{R}} \\
\hline
\eval\left(\texttt{Minus(e1,e2)}, \sigma\right) = (c_1 - c_2) \\
\end{array} (\text{Minus}) $$

$$\begin{array}{c}
\eval(\texttt{e}, \sigma) = c,\ T \in \{ \texttt{Exp}, \texttt{Sine}, \texttt{Cosine} \}, \color{red}{c \in \mathbb{R}}\\
\hline
\eval\left( \texttt{T(e)}, \sigma \right) =  f_{\texttt{T}}(c)
\end{array} (\text{InBuilt-Function-Application})$$

How about __error__? First we will summarize all situations that can produce a value __error__.

$$ \begin{array}{c}
 s \not\in \text{Domain}(\sigma) \\
 \hline
 \eval(\texttt{Ident(s)}, \sigma) = \mathbf{error} \\
 \end{array} (\text{Ident-ERROR}) $$

Other situations are also easy to see:

$$\begin{array}{c}
\eval(\texttt{e1}, \sigma) =  c_1,\ \eval(\texttt{e2}, \sigma) = c_2,\ {c_1 \in \mathbb{R}, c_2 \in \mathbb{R}},\ c_2 = 0\\
\hline
\eval\left(\texttt{Div(e1, e2)}, \sigma\right) = \mathbf{error} \\
\end{array} (\text{Div-ERROR}) $$

Another situation that comes to mind is `Log`.


$$\begin{array}{c}
\eval(\texttt{e}, \sigma) = c,\ {c \in \mathbb{R}},\ {c \leq 0} \\
\hline 
\eval(\texttt{Log(e)}, \sigma) =  \mathbf{error} \\
\end{array}\ \text{(Log-ERROR)}$$

Now we need to write rules that say that once any child of an expression evaluates to an __error__ the expression itself evaluates to an error. This is very cumbersome to do in its fullest exquisite detail. Therefore, we will say so using appropriate notation to help us.

Let us define the set of a __subterms__ of a given term inductively.
- For expressions $e$ of the form `Plus(e1, e2)`, `Minus(e1, e2)`, and
`Mult(e1, e2)` and `Div(e1, e2)`: 
   $$ \text{subterm(e)} = \{ \texttt{e1}, \texttt{e2} \} \cup  \text{subterm}(\texttt{e1}) \cup \text{subterm}(\texttt{e2})$$.
- For expressions $e$ of the form `T(e1)` where `T` can be `Log, Sine, Cosine, Exp`:
$$\text{subterm}(e) = \{ \texttt{e1} \} \cup \text{subterm}(\texttt{e1})$$

__Example:__ Using the definintion, we can show that 
- subterm(`Plus(Const(1.0) , Mult(Const(2.0), Ident("y"))`) is the set { `Const(1.0)`, `Mult(Const(2.0), Ident("y"))`, `Const(2.0)`, `Ident("y")` }


Now we can write a single rule to deal with __error__:

$$\begin{array}{c}
\eval(\texttt{e1}, \sigma) = \mathbf{error},\ \texttt{e1} \in \text{subterm}(e) \\
\hline
\eval( \texttt{e}, \sigma) =  \mathbf{error} \end{array} (\text{Subterm-ERROR})  $$

Let's interpret this rule:
- Premise: under environment $\sigma$ the expression `e1` evaluates to __error__ and `e1` is a subterm of `e`.
- Conclusion: under environment $\sigma$ the expression `e` evalutes to __error__.

The bigstep semantics now clarify how some ambiguous function situations are to be handled.  

### Example 

- Consider the expression `Mult(  Const(0.0), Div(Const(1.0), Const(0.0)))`. We are tempted to  perform a `short circuit rule` as follows:

$$\begin{array}{c}
\eval(\texttt{e1}, \sigma) =  0.0,\\
\hline
\eval(\texttt{Mult(e1, e2)}, \sigma) = 0.0 \end{array} (\text{Shortciruit-Mult})$$


This says that whenever the first argument of multiplication is a zero, let us short circuit the entire computation to zero without even examining them. As you can see this leads to a big problem. What if one of the unexamined terms
causes an error? One can easily slip in such a statement in code for `evalExpr` and no one is any the wiser. The contradiction is only evident when
we take the effort to write the big step semantics.


- What about short circuiting error?

$$ \begin{array}{c}
\eval(\texttt{e1}, \sigma) =  \mathbf{error},\\
\hline
\eval\left(\texttt{Mult(e1, e2)}, \sigma\right) = \mathbf{error} \end{array} (\text{Shortciruit-Mult-ERROR})$$

This is subsumed by the Subterm-ERROR rule already stated. In fact, throwing an exception upon encountering an error seems consistent with the semantics written here especially if we map that exception to the value __error__. This will be entirely consistent with the Subterm-ERROR rule that states that if any subterm evaluates to __error__ the expression as a whole also evaluates to an error.




## From Big-Step Semantics To Code

Note that while big-step semantics are very good at expressing how expressions evaluate to their intended values at a high level, it is not good in expressing issues such as 
- What is the order of evaluation? For instance we would like to say that if we have an expression `Plus(e1,e2)` we evalute this from left to right, starting with `e1` and ending at `e2`.
- How do we deal with _side effects? (currently there are no side effects) and so this aspect of our discussion needs to await a future lecture.

To nail down such issues, we may have to give something called _small step semantics_. However, we will skip small step semantics for now and await such a time when our language has some side effects and mutation, which are both non-issues at present.

For now, let us see how the semantics we have expressed thus far can be translated faithfully into code. Let us make this methodical. We first systematically capture the possible values that an expression can produce. These include the special value `Error` or a `Number`

In [4]:
sealed trait Value
case object Error extends Value
case class Number(c: Double) extends Value

defined trait Value
defined object Error
defined class Number

First we define some convenient functions to compute expected operations over the newly defined values.

In [5]:
// For convenience, we will package everything into an object. 
// This object has the main function evalExpr as well as the helper functions 

// Note: object in scala is different from class. when you declare something as an object
// then there can be exactly one instance of this in the memory and it will be
// called by the same name EvalExprObject.
// Therefore, a function foo in EvalExprObject will be called as EvalExprObject.foo(...)

object EvalExprObject { 

    
        // Function binaryOperation will take two sub-expressions
        // evaluate the subexpressions `e1`, `e2`
        // Check if they are error
        // If not apply a function `resFun` provided by the user to the values from 
        // evaluating `e1`, `e2`
        def binaryOperation(e1: Expr, 
                            e2: Expr, 
                            resFun: (Double, Double) => Value, 
                            env: Map[String, Double] ) = {
            val v1 = this.evalExpr(e1, env) // Evaluate e1 --> recursive
            val v2 = this.evalExpr(e2, env)  // Evaluate e2
            (v1, v2) match {
                    case (_, Error) => Error // If either is Error then result is Error
                    case (Error, _) => Error 
                    case (Number(f1), Number(f2)) => resFun(f1, f2) //Otherwise apply function
                }
            
        }
    
        // Compute Log if the value is not error 
        def logValue(val0: Value): Value = val0 match {
            case Error => Error
            case Number(c1) if c1 > 0.0 => Number( math.log(c1) )
            case _ => Error
        }
        // compute exp if the value is not error
        def expValue(val0: Value): Value = val0 match {
            case Error => Error
            case Number(c1)  => Number( math.exp(c1) )
        }
        // compute sine if the value is not error
        def sineValue(val0: Value): Value = val0 match {
            case Error => Error
            case Number(c1)  => Number( math.sin(c1) )
        }
        // compute cosine if the value is not error 
        def cosineValue(val0: Value): Value = val0 match {
            case Error => Error
            case Number(c1)  => Number( math.cos(c1) )
        }
        def evalExpr (e: Expr, env: Map[String, Double]): Value =  e match { 
            // Split cases on what e can be

            case Const(f) => Number(f) // e is a constant

            // e is an identifier
            case Ident (str) => { if (env.contains(str)){ // str \in domain(env) 
                                        Number( env(str) )  //it is part of the environment
                                    } else {
                                        Error  // it is not part of the environment
                                    }
                                }
            
            // e is of the form Plus of two subexpressions
            case Plus(e1, e2) => {
                // Define the operation of adding two numbers
                def add2(x: Double, y: Double): Value = Number(x + y)
                // User our helper function that was defined already
                this.binaryOperation(e1, e2, add2, env)
            }

            // e is of the form Minus of two subexpressions
            case Minus(e1, e2) => {
                def sub2(x: Double, y: Double): Value = Number(x - y)
                this.binaryOperation(e1, e2, sub2, env)
            }

            // e is of the form multiply two subexpressions
            case Mult(e1, e2) => {
                def mult2(x: Double, y: Double): Value = Number(x * y)
                this.binaryOperation(e1, e2, mult2, env)
            }

            // e is of the form division of two subexpressions
            case Div(e1, e2) => {
                // Carefully define division to handle divide by zero case as well.
                def div2(x: Double, y: Double): Value  = 
                      { if (y != 0.0 ) Number(x/y) else Error }
                //Use the function already defined
                this.binaryOperation(e1, e2, div2, env)
            }

            case Log(e) => this.logValue(evalExpr(e, env))

            case Exp(e) => this.expValue( evalExpr(e, env))

            case Sine(e) => this.sineValue( evalExpr(e, env))

            case Cosine(e) => this.cosineValue(evalExpr(e, env))
    }
    
}

defined object EvalExprObject

Let us create an environment that maps some variables names $x, y, Zzz, w, l$ to some values.

In [6]:
val myEnvironment: Map[String, Double] = Map("x" -> 2.0, 
                                             "y" -> 1.5, 
                                             "Zzz" -> 2.8, 
                                             "w" -> 15.2, 
                                             "l" -> 129.3)

myEnvironment: Map[String, Double] = Map(
  "x" -> 2.0,
  "y" -> 1.5,
  "Zzz" -> 2.8,
  "l" -> 129.3,
  "w" -> 15.2
)

In [7]:
val x = Ident("x")
val y = Ident("y")
val z = Ident("Zzz")
// cos(x) + sin(y) + exp(x - (y+z))
val expr1 = Plus(Cosine(x), Plus(Sine(y), Exp(Minus(x, Plus(y,z)))))

x: Ident = Ident("x")
y: Ident = Ident("y")
z: Ident = Ident("Zzz")
expr1: Plus = Plus(
  Cosine(Ident("x")),
  Plus(Sine(Ident("y")), Exp(Minus(Ident("x"), Plus(Ident("y"), Ident("Zzz")))))
)

In [8]:
EvalExprObject.evalExpr(expr1, myEnvironment)

res7: Value = Number(0.6816069937797158)

In [9]:
val w = Ident("w")
// exp(1.5) + sin(2.0) + cos(1.2*2.4)
val expr2 = Plus( 
               Exp(Const(1.5)), 
               Plus( Sine(Const(2.0)), 
                     Cosine( Mult( 
                                  Const(1.2), 
                                  Const(2.4))
                           )
                   )
             )
//(((w+x*y) - (1.2+w))/(1.5+(w+x)))/2.3
val expr3 = Div(Div(Minus( Plus(w, Mult(x, y)), Plus(Const(1.2), w)) , 
                    Plus(Const(1.5), Plus(w, x) )), Const(2.3))

w: Ident = Ident("w")
expr2: Plus = Plus(
  Exp(Const(1.5)),
  Plus(Sine(Const(2.0)), Cosine(Mult(Const(1.2), Const(2.4))))
)
expr3: Div = Div(
  Div(
    Minus(
      Plus(Ident("w"), Mult(Ident("x"), Ident("y"))),
      Plus(Const(1.2), Ident("w"))
    ),
    Plus(Const(1.5), Plus(Ident("w"), Ident("x")))
  ),
  Const(2.3)
)

In [10]:
EvalExprObject.evalExpr(expr2, myEnvironment)

res9: Value = Number(4.4250071847657715)

In [11]:
EvalExprObject.evalExpr(expr3, myEnvironment)

res10: Value = Number(0.04185073238781681)

In [12]:
val expr4 = Log(Plus(Plus(x, y), Plus(w, Const(-18.7))))
EvalExprObject.evalExpr(expr4, myEnvironment)

val expr5 = Div(Const(1.0), Plus(Plus(x, y), Plus(w, Const(-18.7))))
EvalExprObject.evalExpr(expr5, myEnvironment)

expr4: Log = Log(
  Plus(Plus(Ident("x"), Ident("y")), Plus(Ident("w"), Const(-18.7)))
)
res11_1: Value = Error
expr5: Div = Div(
  Const(1.0),
  Plus(Plus(Ident("x"), Ident("y")), Plus(Ident("w"), Const(-18.7)))
)
res11_3: Value = Error